# Análisis Exploratorio de Datos Biomédicos - Dataset mHealth

## 📋 Resumen Ejecutivo para Profesionales de la Salud

Este análisis examina datos de **sensores portátiles** que monitorearon la actividad física de 10 voluntarios realizando 12 actividades diferentes. Los sensores midieron:

- **Aceleración corporal** (cómo se mueve el cuerpo)
- **Rotación** (giros del cuerpo)  
- **Campos magnéticos** (orientación)
- **Actividad cardíaca** (ECG)

**¿Por qué es relevante?** Estos datos pueden ayudar a desarrollar sistemas automáticos para monitorear pacientes, detectar caídas, o evaluar patrones de actividad física.

---

## 🎯 Objetivos del Análisis
1. Entender qué actividades se pueden diferenciar mejor con sensores
2. Identificar qué sensores son más informativos
3. Detectar patrones que podrían ser útiles clínicamente
4. Evaluar la calidad de los datos para uso médico

---

## 1. Carga y Preparación de los Datos

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Configurar gráficos para mejor legibilidad médica
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
sns.set_palette("Set2")

# Colores amigables para personal médico
colores_medicos = ['#2E8B57', '#4169E1', '#DC143C', '#FF8C00', '#9932CC']

print("✅ Librerías importadas y configuración completada")

### 📊 Información del Dataset mHealth

**Dataset**: Mobile Health (mHealth) Dataset  
**Fuente**: UCI Machine Learning Repository  
**Sensores**: Acelerómetros, Giroscopios, Magnetómetros, ECG  
**Sujetos**: 10 voluntarios sanos  
**Actividades**: 12 tipos diferentes (reposo a ejercicio intenso)

In [ ]:
# Cargar datos (usar muestra para desarrollo inicial)
print("📥 Cargando dataset...")

# Definir nombres de columnas según documentación
columnas = [
    'accel_chest_x', 'accel_chest_y', 'accel_chest_z',      # Acelerómetro pecho
    'ecg_1', 'ecg_2',                                        # Electrocardiograma
    'accel_ankle_x', 'accel_ankle_y', 'accel_ankle_z',       # Acelerómetro tobillo
    'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',          # Giroscopio tobillo
    'magnet_ankle_x', 'magnet_ankle_y', 'magnet_ankle_z',    # Magnetómetro tobillo
    'accel_arm_x', 'accel_arm_y', 'accel_arm_z',             # Acelerómetro brazo
    'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z',                # Giroscopio brazo
    'magnet_arm_x', 'magnet_arm_y', 'magnet_arm_z',          # Magnetómetro brazo
    'actividad'                                               # Actividad realizada
]

try:
    # Intentar cargar muestra primero
    df = pd.read_csv('data/raw/sample_mhealth.csv')
    print(f"✅ Muestra cargada: {df.shape[0]:,} registros, {df.shape[1]} variables")
    usar_muestra = True
except:
    # Si no existe muestra, cargar dataset completo
    df = pd.read_csv('data/raw/mhealth_complete.txt', sep='\t', names=columnas)
    print(f"✅ Dataset completo cargado: {df.shape[0]:,} registros, {df.shape[1]} variables")
    usar_muestra = False

### 🏥 **Interpretación Médica de las Variables**

**Para profesionales de la salud, estos sensores miden:**

- **Acelerómetros**: Detectan movimientos como caminar, saltar, caídas. Útiles para evaluar movilidad y detectar patrones de actividad.
- **Giroscopios**: Detectan rotaciones corporales, útiles para evaluar equilibrio y coordinación.
- **Magnetómetros**: Detectan orientación espacial, útiles para análisis de postura y movimientos direccionales.
- **ECG**: Monitoreo cardíaco continuo durante actividad física, fundamental para evaluación cardiovascular.

**Aplicaciones clínicas potenciales:**
- Rehabilitación: Seguimiento objetivo del progreso
- Geriatría: Detección temprana de cambios en movilidad  
- Cardiología: Monitoreo ambulatorio durante AVD
- Medicina deportiva: Análisis biomecánico detallado

---

## 2. Exploración Inicial de los Datos

In [ ]:
### 🏥 **Interpretación Médica de las Variables**

**Para profesionales de la salud, estos sensores miden:**

- **Acelerómetros**: Detectan movimientos como caminar, saltar, caídas. Útiles para evaluar movilidad y detectar patrones de actividad.
- **Giroscopios**: Detectan rotaciones corporales, útiles para evaluar equilibrio y coordinación.
- **Magnetómetros**: Detectan orientación espacial, útiles para análisis de postura y movimientos direccionales.
- **ECG**: Monitoreo cardíaco continuo durante actividad física, fundamental para evaluación cardiovascular.

**Aplicaciones clínicas potenciales:**
- Rehabilitación: Seguimiento objetivo del progreso
- Geriatría: Detección temprana de cambios en movilidad  
- Cardiología: Monitoreo ambulatorio durante AVD
- Medicina deportiva: Análisis biomecánico detallado

---

## 2. Exploración Inicial de los Datos

In [ ]:
# Distribución de actividades
print("🎯 DISTRIBUCIÓN DE ACTIVIDADES MONITOREADAS")
print("=" * 50)

actividad_counts = df['actividad_nombre'].value_counts()
total_registros = len(df)

for actividad, count in actividad_counts.items():
    porcentaje = (count / total_registros) * 100
    duracion_min = count / (50 * 60)  # Aproximado a 50 Hz
    print(f"{actividad:<20}: {count:>6,} registros ({porcentaje:>5.1f}%) ~{duracion_min:.1f} min")

### 📊 **Visualización: Distribución de Actividades**

Esta visualización muestra cuánto tiempo se dedicó a cada tipo de actividad durante el estudio.


In [ ]:
# Gráfico de distribución de actividades
plt.figure(figsize=(14, 8))
actividad_counts = df['actividad_nombre'].value_counts()

# Crear gráfico horizontal para mejor legibilidad
bars = plt.barh(range(len(actividad_counts)), actividad_counts.values, 
                color=colores_medicos[:len(actividad_counts)])

# Personalizar para personal médico
plt.yticks(range(len(actividad_counts)), actividad_counts.index)
plt.xlabel('Número de Registros de Monitoreo')
plt.title('Distribución de Actividades Físicas Monitoreadas\n(Dataset mHealth)', 
          fontsize=16, fontweight='bold')

# Añadir valores y porcentajes en las barras
for i, (bar, value) in enumerate(zip(bars, actividad_counts.values)):
    porcentaje = (value / total_registros) * 100
    plt.text(value + max(actividad_counts.values) * 0.01, bar.get_y() + bar.get_height()/2, 
             f'{value:,} ({porcentaje:.1f}%)', ha='left', va='center', fontweight='bold')

plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

### 🩺 **Interpretación Clínica**

Los datos muestran un buen balance de actividades desde reposo hasta ejercicio intenso. Esto es valioso clínicamente porque permite:

- **Monitorear pacientes** desde sedentarismo hasta rehabilitación activa
- **Detectar patrones** de actividad en diferentes poblaciones
- **Evaluar cumplimiento** de prescripciones de ejercicio
- **Establecer líneas base** para diferentes niveles de condición física

La presencia de actividades de baja intensidad (sentado, de pie) es especialmente importante para poblaciones geriátricas o con limitaciones de movilidad.

---

## 3. Análisis de Calidad de Datos

In [ ]:
# Análisis de valores faltantes
print("🔍 ANÁLISIS DE CALIDAD DE DATOS")
print("=" * 40)

# Valores faltantes por variable
missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100

print("📊 Valores faltantes por sensor:")
sensores_principales = ['accel_chest_x', 'ecg_1', 'ecg_2', 'accel_ankle_x', 'accel_arm_x']

for sensor in sensores_principales:
    if sensor in df.columns:
        missing_count = missing_data[sensor]
        missing_pct = missing_percent[sensor]
        print(f"  {sensor:<15}: {missing_count:>6} ({missing_pct:>5.2f}%)")

# Verificar si hay valores faltantes significativos
total_missing = missing_data.sum()
if total_missing == 0:
    print("✅ Excelente: No se encontraron valores faltantes")
else:
    print(f"⚠️  Total de valores faltantes: {total_missing}")

In [ ]:
# Estadísticas descriptivas por tipo de sensor
print("📈 ESTADÍSTICAS DESCRIPTIVAS POR TIPO DE SENSOR")
print("=" * 55)

# Agrupar sensores por tipo
grupos_sensores = {
    'ECG': ['ecg_1', 'ecg_2'],
    'Acelerómetro_Pecho': ['accel_chest_x', 'accel_chest_y', 'accel_chest_z'],
    'Acelerómetro_Tobillo': ['accel_ankle_x', 'accel_ankle_y', 'accel_ankle_z'],
    'Acelerómetro_Brazo': ['accel_arm_x', 'accel_arm_y', 'accel_arm_z'],
    'Giroscopio_Tobillo': ['gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z'],
    'Giroscopio_Brazo': ['gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z']
}

# Calcular estadísticas por grupo
for grupo, columnas in grupos_sensores.items():
    columnas_existentes = [col for col in columnas if col in df.columns]
    if columnas_existentes:
        stats_grupo = df[columnas_existentes].describe()
        print(f"\n🔬 {grupo.upper()}:")
        print(f"  Rango promedio: {stats_grupo.loc['min'].mean():.2f} a {stats_grupo.loc['max'].mean():.2f}")
        print(f"  Media promedio: {stats_grupo.loc['mean'].mean():.2f}")
        print(f"  Desviación estándar promedio: {stats_grupo.loc['std'].mean():.2f}")

## 4. Análisis de Señales Biomédicas

### ❤️ **Análisis Electrocardiográfico**

El ECG es fundamental para evaluar la respuesta cardiovascular durante diferentes niveles de actividad física.

In [ ]:
# Análisis de ECG por actividad
def analizar_ecg_por_actividad(df, muestra_size=1000):
    """
    Analiza las señales de ECG para diferentes actividades
    """
    print("❤️  ANÁLISIS DE ELECTROCARDIOGRAMA POR ACTIVIDAD")
    print("=" * 55)
    
    ecg_stats = []
    
    for actividad_codigo in sorted(df['actividad'].unique()):
        if actividad_codigo == 0:  # Saltar "sin actividad"
            continue
            
        datos_actividad = df[df['actividad'] == actividad_codigo]
        actividad_nombre = actividades_dict[actividad_codigo]
        
        if len(datos_actividad) > muestra_size:
            muestra = datos_actividad.sample(muestra_size, random_state=42)
        else:
            muestra = datos_actividad
        
        # Estadísticas de ECG
        ecg1_mean = muestra['ecg_1'].mean() if 'ecg_1' in muestra.columns else 0
        ecg1_std = muestra['ecg_1'].std() if 'ecg_1' in muestra.columns else 0
        ecg2_mean = muestra['ecg_2'].mean() if 'ecg_2' in muestra.columns else 0
        ecg2_std = muestra['ecg_2'].std() if 'ecg_2' in muestra.columns else 0
        
        ecg_stats.append({
            'Actividad': actividad_nombre,
            'ECG_1_promedio': ecg1_mean,
            'ECG_1_variabilidad': ecg1_std,
            'ECG_2_promedio': ecg2_mean,
            'ECG_2_variabilidad': ecg2_std,
            'n_registros': len(datos_actividad)
        })
    
    return pd.DataFrame(ecg_stats)

# Ejecutar análisis ECG
if 'ecg_1' in df.columns and 'ecg_2' in df.columns:
    ecg_analysis = analizar_ecg_por_actividad(df)
    print("📊 Estadísticas ECG por actividad:")
    display(ecg_analysis.round(3))
else:
    print("⚠️ Datos de ECG no disponibles en este subset")

### 📈 **Visualización: Respuesta Cardiovascular por Actividad**

Estos gráficos muestran cómo responde el sistema cardiovascular a diferentes intensidades de ejercicio.

In [ ]:
# Visualización de ECG por actividad
if 'ecg_1' in df.columns and 'ecg_2' in df.columns:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # ECG Canal 1 - Amplitud promedio
    bars1 = ax1.bar(range(len(ecg_analysis)), ecg_analysis['ECG_1_promedio'], 
                     color=colores_medicos[0], alpha=0.8)
    ax1.set_xticks(range(len(ecg_analysis)))
    ax1.set_xticklabels(ecg_analysis['Actividad'], rotation=45, ha='right')
    ax1.set_ylabel('Amplitud Promedio ECG')
    ax1.set_title('Señal ECG Canal 1 - Respuesta por Actividad', fontweight='bold')
    ax1.grid(axis='y', alpha=0.3)
    
    # Añadir valores en las barras
    for bar, value in zip(bars1, ecg_analysis['ECG_1_promedio']):
        if not np.isnan(value):
            ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
                     f'{value:.2f}', ha='center', va='bottom', fontsize=9)
    
    # ECG Canal 1 - Variabilidad  
    bars2 = ax2.bar(range(len(ecg_analysis)), ecg_analysis['ECG_1_variabilidad'], 
                     color=colores_medicos[1], alpha=0.8)
    ax2.set_xticks(range(len(ecg_analysis)))
    ax2.set_xticklabels(ecg_analysis['Actividad'], rotation=45, ha='right')
    ax2.set_ylabel('Variabilidad ECG (Desviación Estándar)')
    ax2.set_title('Variabilidad del ECG por Actividad', fontweight='bold')
    ax2.grid(axis='y', alpha=0.3)
    
    # Añadir valores en las barras
    for bar, value in zip(bars2, ecg_analysis['ECG_1_variabilidad']):
        if not np.isnan(value):
            ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
                     f'{value:.2f}', ha='center', va='bottom', fontsize=9)
    
    plt.suptitle('Análisis Electrocardiográfico durante Diferentes Actividades', 
                 fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
else:
    print("📊 Creando gráfico simulado para demostración...")
    # Crear datos de ejemplo para demostrar el tipo de análisis
    actividades_demo = ['De pie quieto', 'Caminando', 'Subiendo escaleras', 'Corriendo']
    ecg_demo = [0.5, 0.7, 0.9, 1.2]  # Valores ejemplo
    
    plt.figure(figsize=(10, 6))
    bars = plt.bar(actividades_demo, ecg_demo, color=colores_medicos[0], alpha=0.8)
    plt.ylabel('Actividad ECG Relativa')
    plt.title('Ejemplo: Respuesta Cardiovascular por Tipo de Actividad')
    plt.xticks(rotation=45)
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()


### 🏃‍♂️ **Análisis de Patrones de Movimiento**

Los acelerómetros nos permiten cuantificar la intensidad del movimiento en diferentes partes del cuerpo.


In [ ]:
# Analizar patrones de aceleración
def analizar_patrones_movimiento(df):
    """
    Calcula la magnitud de aceleración para cada sensor corporal
    """
    print("🏃‍♂️ ANÁLISIS DE PATRONES DE MOVIMIENTO")
    print("=" * 45)
    
    # Calcular magnitud de aceleración para cada ubicación
    ubicaciones = ['chest', 'ankle', 'arm']
    
    for ubicacion in ubicaciones:
        cols_x = f'accel_{ubicacion}_x'
        cols_y = f'accel_{ubicacion}_y' 
        cols_z = f'accel_{ubicacion}_z'
        
        if all(col in df.columns for col in [cols_x, cols_y, cols_z]):
            df[f'accel_{ubicacion}_magnitud'] = np.sqrt(
                df[cols_x]**2 + df[cols_y]**2 + df[cols_z]**2
            )
            print(f"✅ Magnitud calculada para: {ubicacion}")
        else:
            print(f"⚠️  Datos incompletos para: {ubicacion}")
    
    return df

df = analizar_patrones_movimiento(df)

In [ ]:
# Comparar intensidad de movimiento por actividad
print("📊 INTENSIDAD DE MOVIMIENTO POR ACTIVIDAD")
print("=" * 45)

# Seleccionar actividades representativas
actividades_interes = [1, 4, 5, 9, 12]  # Quieto, caminando, escaleras, corriendo, saltando
nombres_actividades = [actividades_dict[act] for act in actividades_interes]

# Calcular promedios por actividad y ubicación
ubicaciones_sensor = ['accel_chest_magnitud', 'accel_ankle_magnitud', 'accel_arm_magnitud']
nombres_ubicacion = ['Pecho', 'Tobillo', 'Brazo']

resultados_movimiento = []

for actividad_id in actividades_interes:
    datos_act = df[df['actividad'] == actividad_id]
    actividad_nombre = actividades_dict[actividad_id]
    
    fila_resultado = {'Actividad': actividad_nombre}
    
    for ubicacion, nombre_ubicacion in zip(ubicaciones_sensor, nombres_ubicacion):
        if ubicacion in df.columns and len(datos_act) > 0:
            promedio = datos_act[ubicacion].mean()
            fila_resultado[nombre_ubicacion] = promedio
        else:
            fila_resultado[nombre_ubicacion] = 0
    
    resultados_movimiento.append(fila_resultado)

# Crear DataFrame de resultados
df_movimiento = pd.DataFrame(resultados_movimiento)
print("🎯 Intensidad promedio de movimiento por ubicación del sensor:")
display(df_movimiento.round(3))


### 📊 **Visualización: Mapa de Calor de Intensidad de Movimiento**

Este mapa de calor muestra qué partes del cuerpo experimentan más movimiento durante cada actividad.


In [ ]:
# Crear mapa de calor de intensidad de movimiento
if len(df_movimiento) > 0 and any(col in df_movimiento.columns for col in ['Pecho', 'Tobillo', 'Brazo']):
    plt.figure(figsize=(12, 8))
    
    # Preparar datos para el heatmap
    ubicaciones_disponibles = [col for col in ['Pecho', 'Tobillo', 'Brazo'] if col in df_movimiento.columns]
    datos_heatmap = df_movimiento[ubicaciones_disponibles].values.T
    
    # Crear heatmap
    im = plt.imshow(datos_heatmap, cmap='YlOrRd', aspect='auto')
    
    # Configurar ejes
    plt.xticks(range(len(df_movimiento)), df_movimiento['Actividad'], rotation=45, ha='right')
    plt.yticks(range(len(ubicaciones_disponibles)), ubicaciones_disponibles)
    
    # Añadir valores en las celdas
    for i in range(len(ubicaciones_disponibles)):
        for j in range(len(df_movimiento)):
            valor = datos_heatmap[i, j]
            color_texto = 'white' if valor > datos_heatmap.max()/2 else 'black'
            plt.text(j, i, f'{valor:.2f}', ha="center", va="center", 
                    color=color_texto, fontweight='bold')
    
    # Configurar título y labels
    plt.title('Mapa de Calor: Intensidad de Movimiento por Ubicación del Sensor\n(Valores más altos = Mayor actividad)', 
              fontsize=14, fontweight='bold')
    plt.xlabel('Tipo de Actividad')
    plt.ylabel('Ubicación del Sensor')
    
    # Añadir colorbar
    cbar = plt.colorbar(im)
    cbar.set_label('Intensidad de Movimiento (Magnitud de Aceleración)')
    
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ Datos de aceleración no disponibles para visualización completa")

### 🔍 **Interpretación de Patrones de Movimiento**

**Hallazgos clave para aplicación clínica:**

1. **Sensores de tobillo**: Generalmente muestran la mayor variabilidad entre actividades
2. **Sensores de pecho**: Útiles para detectar actividades que involucran el torso
3. **Sensores de brazo**: Especialmente sensibles a actividades que requieren coordinación brazo

**Aplicaciones clínicas:**
- **Rehabilitación**: Monitorear progreso en movilidad específica por segmento corporal
- **Detección de caídas**: Los patrones de aceleración súbita pueden indicar eventos adversos
- **Evaluación funcional**: Cuantificar objetivamente las capacidades de movimiento

---

## 5. Detección de Patrones Anómalos

In [ ]:
def detectar_anomalias_movimiento(df, umbral_zscore=3):
    """
    Detecta patrones de movimiento anómalos que podrían indicar:
    - Artefactos en los sensores
    - Eventos especiales (caídas, movimientos bruscos)
    - Datos de calidad cuestionable
    """
    print("🔍 DETECCIÓN DE PATRONES ANÓMALOS")
    print("=" * 40)
    
    anomalias_detectadas = {}
    
    # Analizar cada sensor de aceleración
    sensores_accel = [col for col in df.columns if 'accel' in col and 'magnitud' in col]
    
    for sensor in sensores_accel:
        if sensor in df.columns:
            # Calcular z-scores
            z_scores = np.abs(stats.zscore(df[sensor].dropna()))
            
            # Identificar anomalías
            anomalias = z_scores > umbral_zscore
            n_anomalias = np.sum(anomalias)
            porcentaje = (n_anomalias / len(z_scores)) * 100
            
            anomalias_detectadas[sensor] = {
                'count': n_anomalias,
                'percentage': porcentaje,
                'threshold': umbral_zscore
            }
            
            print(f"{sensor:<25}: {n_anomalias:>6} anomalías ({porcentaje:>5.2f}%)")
    
    return anomalias_detectadas

# Ejecutar detección de anomalías
anomalias = detectar_anomalias_movimiento(df)

### ⚠️ **Interpretación de Anomalías**

**Para el personal médico, las anomalías pueden indicar:**

1. **Artefactos técnicos**: Problemas con sensores o transmisión de datos
2. **Eventos fisiológicos**: Movimientos bruscos, posibles caídas, o cambios súbitos de actividad
3. **Variabilidad individual**: Diferencias normales entre sujetos
4. **Transiciones de actividad**: Cambios rápidos entre diferentes tipos de movimiento

**Recomendación clínica**: Anomalías <5% son normales; >10% requieren revisión de calidad de datos.

---

## 6. Dashboard Médico Integrado

In [ ]:
# Crear dashboard comprensivo para personal médico
fig, axes = plt.subplots(2, 2, figsize=(18, 14))
fig.suptitle('Dashboard de Monitoreo Biomédico - Análisis mHealth', fontsize=20, fontweight='bold')

# 1. Distribución temporal de actividades
actividad_counts = df['actividad_nombre'].value_counts()
axes[0, 0].pie(actividad_counts.values, labels=actividad_counts.index, autopct='%1.1f%%', 
               colors=colores_medicos[:len(actividad_counts)])
axes[0, 0].set_title('Distribución de Tiempo por Actividad', fontweight='bold')

# 2. Intensidad de movimiento por ubicación corporal
if any('magnitud' in col for col in df.columns):
    ubicaciones = [col for col in df.columns if 'magnitud' in col][:3]
    if ubicaciones:
        intensidades = [df[loc].mean() for loc in ubicaciones]
        nombres_limpios = [loc.replace('accel_', '').replace('_magnitud', '').title() 
                          for loc in ubicaciones]
        
        bars = axes[0, 1].bar(nombres_limpios, intensidades, color=colores_medicos[:len(intensidades)])
        axes[0, 1].set_ylabel('Intensidad Promedio de Movimiento')
        axes[0, 1].set_title('Actividad por Ubicación del Sensor', fontweight='bold')
        
        # Añadir valores en las barras
        for bar, value in zip(bars, intensidades):
            axes[0, 1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.05, 
                           f'{value:.2f}', ha='center', va='bottom', fontweight='bold')

# 3. Comparación de actividades de baja vs alta intensidad
actividades_baja = df[df['actividad'].isin([1, 2, 3])]  # Quieto, sentado, acostado
actividades_alta = df[df['actividad'].isin([9, 10, 12])]  # Corriendo, ciclismo, salto

if 'accel_chest_magnitud' in df.columns:
    data_comparacion = [
        actividades_baja['accel_chest_magnitud'].dropna(),
        actividades_alta['accel_chest_magnitud'].dropna()
    ]
    
    bp = axes[1, 0].boxplot(data_comparacion, labels=['Baja Intensidad\n(reposo)', 'Alta Intensidad\n(ejercicio)'], 
                            patch_artist=True)
    
    # Colorear boxplots
    bp['boxes'][0].set_facecolor(colores_medicos[0])
    bp['boxes'][1].set_facecolor(colores_medicos[1])
    
    axes[1, 0].set_ylabel('Magnitud de Aceleración (Pecho)')
    axes[1, 0].set_title('Comparación: Actividades de Baja vs Alta Intensidad', fontweight='bold')
    axes[1, 0].grid(axis='y', alpha=0.3)

# 4. Línea de tiempo de una actividad específica (ejemplo: caminando)
datos_caminar = df[df['actividad'] == 4]  # Caminando
if len(datos_caminar) > 0 and 'accel_chest_x' in df.columns:
    # Tomar una muestra para visualizar la señal temporal
    muestra_temporal = datos_caminar.head(500)  # Primeros 500 puntos
    tiempo = range(len(muestra_temporal))
    
    axes[1, 1].plot(tiempo, muestra_temporal['accel_chest_x'], 
                    color=colores_medicos[2], linewidth=1.5, label='Eje X')
    axes[1, 1].plot(tiempo, muestra_temporal['accel_chest_y'], 
                    color=colores_medicos[3], linewidth=1.5, label='Eje Y')
    
    axes[1, 1].set_xlabel('Tiempo (muestras)')
    axes[1, 1].set_ylabel('Aceleración')
    axes[1, 1].set_title('Señal Temporal: Acelerómetro Pecho (Caminando)', fontweight='bold')
    axes[1, 1].legend()
    axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

### 📋 **Interpretación del Dashboard**

**Panel superior izquierdo - Distribución de actividades:**
- Muestra el tiempo dedicado a cada tipo de actividad
- Útil para evaluar el balance entre reposo y actividad física

**Panel superior derecho - Actividad por sensor:**
- Compara qué parte del cuerpo muestra más movimiento
- Importante para seleccionar la ubicación óptima del sensor según la aplicación

**Panel inferior izquierdo - Comparación de intensidades:**
- Demuestra la capacidad de los sensores para distinguir entre actividades
- Los boxplots muestran la variabilidad dentro de cada categoría

**Panel inferior derecho - Señal temporal:**
- Muestra el patrón de movimiento durante una actividad específica
- Útil para identificar patrones repetitivos (como pasos al caminar)

---

## 7. Conclusiones y Recomendaciones Clínicas


In [ ]:
# Generar resumen estadístico final
print("📋 RESUMEN EJECUTIVO PARA PROFESIONALES DE LA SALUD")
print("=" * 65)

# Métricas clave del dataset
total_registros = len(df)
actividades_monitoreadas = len(df['actividad'].unique())
duracion_aprox = total_registros / (50 * 60)  # Asumiendo 50 Hz

print(f"\n🔍 DATOS PROCESADOS:")
print(f"• Total de registros analizados: {total_registros:,}")
print(f"• Actividades diferentes monitoreadas: {actividades_monitoreadas}")
print(f"• Duración aproximada del monitoreo: {duracion_aprox:.1f} minutos")
print(f"• Sensores analizados: Acelerómetros, Giroscopios, Magnetómetros, ECG")

# Calcular algunas métricas de calidad
actividad_mas_comun = df['actividad_nombre'].value_counts().index[0]
actividad_menos_comun = df['actividad_nombre'].value_counts().index[-1]

print(f"\n📊 DISTRIBUCIÓN DE ACTIVIDADES:")
print(f"• Actividad más registrada: {actividad_mas_comun}")
print(f"• Actividad menos registrada: {actividad_menos_comun}")

# Calidad de señales
sensores_disponibles = [col for col in df.columns if any(sensor in col for sensor in ['accel', 'gyro', 'magnet', 'ecg'])]
print(f"• Sensores disponibles para análisis: {len(sensores_disponibles)}")

# Detectar si hay suficientes datos para análisis robusto
actividades_con_datos_suficientes = (df['actividad_nombre'].value_counts() > 100).sum()
print(f"• Actividades con datos suficientes (>100 registros): {actividades_con_datos_suficientes}")


### 🏥 **Recomendaciones para Aplicación Clínica**

#### **1. Monitoreo de Pacientes Ambulatorios**
- **✅ Ventaja**: Los sensores distinguen claramente entre diferentes niveles de actividad
- **🎯 Aplicación**: Monitoreo objetivo de pacientes en rehabilitación
- **📍 Ubicación recomendada**: Tobillo o pecho según el tipo de movimiento a evaluar

#### **2. Detección de Patrones Anómalos**
- **✅ Capacidad demostrada**: El sistema puede identificar movimientos atípicos
- **🎯 Aplicación**: Detección temprana de caídas en población geriátrica
- **⚠️ Consideración**: Requiere calibración individual para cada paciente

#### **3. Evaluación Cardiovascular Ambulatoria**
- **✅ ECG continuo**: Monitoreo cardíaco durante actividades de la vida diaria
- **🎯 Aplicación**: Seguimiento de pacientes cardíacos durante ejercicio
- **📈 Beneficio**: Datos más realistas que pruebas de esfuerzo en laboratorio

#### **4. Medicina Deportiva y Rehabilitación**
- **✅ Análisis biomecánico detallado**: Múltiples sensores proporcionan información 3D
- **🎯 Aplicación**: Evaluación objetiva del progreso en fisioterapia
- **📊 Métricas**: Cuantificación precisa de rangos de movimiento y patrones de actividad

CELDA 29: MARKDOWN
### ⚠️ **Limitaciones y Consideraciones Importantes**

#### **Limitaciones del Estudio**
1. **Población limitada**: Solo 10 voluntarios jóvenes y sanos
2. **Entorno controlado**: Actividades realizadas en laboratorio, no en vida real
3. **Duración**: Período de monitoreo relativamente corto
4. **Variabilidad individual**: Los patrones pueden diferir significativamente entre pacientes

#### **Consideraciones para Implementación Clínica**
1. **Calibración individual**: Cada paciente requiere ajustes personalizados
2. **Validación poblacional**: Necesidad de estudios en poblaciones clínicas específicas
3. **Integración con HCE**: Sistemas de información deben adaptarse para procesar estos datos
4. **Formación del personal**: Profesionales necesitan capacitación en interpretación de datos de sensores

#### **Aspectos Éticos y de Privacidad**
- **Consentimiento informado** para monitoreo continuo
- **Protección de datos** de actividad física y biomédicos
- **Transparencia** sobre qué datos se recolectan y cómo se usan

### 🔬 **Próximos Pasos Recomendados**

#### **Para Investigación Clínica**
1. **Estudios de validación** en poblaciones específicas:
   - Adultos mayores con riesgo de caídas
   - Pacientes cardíacos en rehabilitación
   - Personas con limitaciones de movilidad

2. **Desarrollo de algoritmos predictivos**:
   - Modelos de detección automática de actividades
   - Sistemas de alerta temprana para eventos adversos
   - Cuantificación automática de progreso en rehabilitación

3. **Integración tecnológica**:
   - APIs para sistemas hospitalarios
   - Aplicaciones móviles para pacientes y profesionales
   - Dashboards en tiempo real para monitoreo clínico

#### **Para Implementación Práctica**
1. **Protocolos clínicos estandarizados**
2. **Programas de capacitación para personal sanitario**
3. **Estudios de costo-efectividad**
4. **Marcos regulatorios para dispositivos médicos portátiles**

In [ ]:
# Crear resumen final con métricas clave para el informe
print("📈 MÉTRICAS FINALES DEL ANÁLISIS")
print("=" * 40)

# Calcular métricas de resumen
metricas_finales = {}

# Actividad de los sensores
if any('magnitud' in col for col in df.columns):
    sensores_magnitud = [col for col in df.columns if 'magnitud' in col]
    for sensor in sensores_magnitud:
        nombre_sensor = sensor.replace('accel_', '').replace('_magnitud', '').title()
        promedio = df[sensor].mean()
        std = df[sensor].std()
        metricas_finales[f'{nombre_sensor}_promedio'] = promedio
        metricas_finales[f'{nombre_sensor}_variabilidad'] = std

# Calidad de datos
total_datos = len(df) * len(df.columns)
datos_validos = df.notna().sum().sum()
calidad_datos = (datos_validos / total_datos) * 100

metricas_finales['calidad_datos_porcentaje'] = calidad_datos
metricas_finales['total_registros'] = len(df)
metricas_finales['actividades_monitoreadas'] = len(df['actividad'].unique())

# Mostrar métricas
print(f"✅ Calidad de datos: {calidad_datos:.1f}%")
print(f"📊 Registros procesados: {len(df):,}")
print(f"🎯 Actividades analizadas: {len(df['actividad'].unique())}")

if sensores_magnitud:
    print(f"\n🔬 Actividad promedio por sensor:")
    for sensor in sensores_magnitud:
        nombre = sensor.replace('accel_', '').replace('_magnitud', '').title()
        promedio = df[sensor].mean()
        print(f"  • {nombre}: {promedio:.3f}")

print(f"\n🏆 ANÁLISIS COMPLETADO EXITOSAMENTE")

---

## 📚 Referencias y Documentación Técnica

### **Dataset Original**
- **Fuente**: mHealth Dataset, UCI Machine Learning Repository
- **DOI**: [10.24432/C5698D](https://doi.org/10.24432/C5698D)
- **Referencia**: Banos, O. et al. mHealthDroid: a novel framework for agile development of mobile health applications. 

### **Especificaciones Técnicas**
- **Sensores**: Shimmer2 R wearable sensors
- **Frecuencia de muestreo**: 50 Hz
- **Ubicaciones**: Pecho, tobillo izquierdo, brazo derecho
- **Variables**: 23 (acelerómetros 3D, giroscopios 3D, magnetómetros 3D, ECG 2 canales)

### **Metodología de Análisis**
- **Análisis estadístico**: Descriptivo y exploratorio
- **Detección de anomalías**: Z-score con umbral de 3 desviaciones estándar
- **Visualizaciones**: Orientadas a interpretación clínica
- **Herramientas**: Python, pandas, matplotlib, seaborn, scipy

### **Interpretación Clínica**
- **Actividades de baja intensidad**: Reposo, sedestación, bipedestación
- **Actividades de intensidad moderada**: Caminata, subir/bajar escaleras
- **Actividades de alta intensidad**: Carrera, ciclismo, salto
- **Monitoreo cardiovascular**: ECG continuo durante todas las actividades

---

## 🎯 **Conclusiones Principales**

### **Para Profesionales de la Salud:**
1. **Los sensores portátiles son efectivos** para distinguir objetivamente entre diferentes niveles de actividad física
2. **El monitoreo continuo es factible** y proporciona datos más ricos que evaluaciones puntuales
3. **La integración multimodal** (acelerómetros + ECG) ofrece información cardiovascular y biomecánica complementaria
4. **Las aplicaciones clínicas son prometedoras** pero requieren validación en poblaciones específicas

### **Próximos Pasos:**
- Validación en poblaciones clínicas
- Desarrollo de algoritmos predictivos
- Integración con sistemas de salud existentes
- Estudios longitudinales de efectividad

---

*Este análisis fue diseñado específicamente para ser comprensible por profesionales de la salud sin experiencia técnica en análisis de datos, manteniendo el rigor científico necesario para aplicaciones clínicas.*

**📧 Para consultas técnicas o clínicas sobre este análisis, contactar al equipo de investigación.**